<a href="https://colab.research.google.com/github/albuut/Medical-Malpractice-Analysis/blob/Google-Collab-Edit/CS203_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Malpractice Analysis



# Google Colab Startup
Make sure to clone the repository into the Google Colab.

In [ ]:
!git clone https://github.com/albuut/Medical-Malpractice-Analysis.git

Run the following code.

In [ ]:
!python /content/Medical-Malpractice-Analysis/bootup.py

# Data Cleaning

Data manipulated from the origional data set

1. Amount: Log(Amount)

2. Speciality: Dummy variable encoded into different specialities.

3. Insurance: Dummy variable enconded into different insurances.

4. Marital Status: Dummy variabled econded into different marital status.

5. Gender: One hot encoded gender for Male = 0 and Female = 1.




We can use the following lines to generate the cleaned up data from csv file with different log values down below.

In [ ]:
!python process_data.py medicalmalpractice.csv log10_data.csv 10
!python process_data.py medicalmalpractice.csv log2_data.csv 2
!python process_data.py medicalmalpractice.csv ln_data.csv ln

We then display the origonal data down below.

In [29]:
import pandas as pd
df_origional = pd.read_csv('medicalmalpractice.csv')
df_origional

,Amount,Severity,Age,Private Attorney,Marital Status,Specialty,Insurance,Gender
0,57041,7,62,1,2,Family Practice,Private,Male
1,324976,6,38,1,2,OBGYN,No Insurance,Female
2,135383,4,34,1,2,Cardiology,Unknown,Male
3,829742,7,42,1,1,Pediatrics,No Insurance,Female
4,197675,3,60,0,2,OBGYN,Medicare/Medicaid,Female
...,...,...,...,...,...,...,...,...
79205,25305,4,64,1,2,General Surgery,Unknown,Male
79206,43098,3,87,0,4,Orthopedic Surgery,Unknown,Male
79207,35398,3,81,0,4,Anesthesiology,Unknown,Male
79208,154228,9,19,0,4,Dermatology,Unknown,Female


We can then observe how the data is cleaned and seperated into different variables to be used in different forms of regression or analysis.

In [30]:
import pandas as pd
df_log10 = pd.read_csv('log10_data.csv')
df_log10

,Amount,log_Amount,Severity,Age,Private Attorney,Gender,Anesthesiology,Cardiology,Dermatology,Emergency Medicine,...,Medicare/Medicaid,No Insurance,Private,Unknown,Workers Compensation,Divorced,Single,Married,Widowed,Unknown.1
0,57041,4.756187,7,62,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,324976,5.511851,6,38,1,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,135383,5.131564,4,34,1,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
3,829742,5.918943,7,42,1,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,197675,5.295952,3,60,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79205,25305,4.403206,4,64,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
79206,43098,4.634457,3,87,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
79207,35398,4.548979,3,81,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
79208,154228,5.188163,9,19,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
